Load the training and test data.

In [209]:
"""
Load the training and test data
"""

import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder, LabelEncoder

# Specify data and output directories
main ='/Users/kmontgomery/Documents/git/article-information-2019/data/output/'
output = '/Users/kmontgomery/Documents/information_article/'
#filename = 'hmda_lar_2018_orig_mtg_sample.csv'
train_filename = 'hmda_train.csv'
test_filename = 'hmda_test.csv'

TRAIN = pd.read_csv(main + train_filename)
TEST = pd.read_csv(main + test_filename)

training_columns = list(TRAIN.columns)
target_column = 'high_priced'

In [211]:
"""
List columns with missing values.
"""

missing_value_columns = ["loan_to_value_ratio",  "discount_points", "lender_credits"]
missing_value_columns += ["loan_term", "prepayment_penalty_term", "intro_rate_period"]
missing_value_columns += ["property_value", "income", "debt_to_income_ratio"]
missing_value_columns += ["loan_to_value_ratio", "discount_points", "lender_credits", "loan_term"]
missing_value_columns += ["prepayment_penalty_term", "intro_rate_period", "property_value", "income"]
missing_value_columns += ["debt_to_income_ratio"]

In [212]:
"""
Flag missing columns
"""

def is_missing(x):
    return int(np.isnan(x))


In [213]:
"""
Fill in missing values and create missing values.
"""

for col in missing_value_columns:
    TRAIN[col + '_is_missing'] = TRAIN[col].isnull().apply(int)
    TRAIN[col] = TRAIN[col].fillna(-1)
    TEST[col + '_is_missing'] = TEST[col].isnull().apply(int)
    TEST[col] =  TEST[col].fillna(-1)    
      

In [214]:
# List of columns to drop
drop_columns = ["loan_purpose",  "loan_purpose_desc", "reverse_mortgage_desc", "negative_amortization_desc", "interest_only_payment_desc" ]
drop_columns += ["balloon_payment_desc", "applicant_credit_score_type_desc", "co_applicant_credit_score_type_desc"]

In [215]:
# List of protected columns
protected_columns = ["derived_ethnicity", "derived_race", "derived_sex"]

In [216]:
# List of columns to be target encoded

target_encoded = []

In [217]:
# List of columns to be one hot encoded

one_hot = ["derived_loan_product_type", "reverse_mortgage", "negative_amortization"]
one_hot += ["interest_only_payment", "balloon_payment", "applicant_credit_score_type"]
one_hot += [ "co_applicant_credit_score_type", "applicant_age", "co_applicant_age"]
one_hot += ["applicant_age_above_62", "co_applicant_age_above_62" ]
one_hot += ["derived_ethnicity",  "derived_race", "derived_sex", "debt_to_income_ratio"]



'\none_hot = ["conforming_loan_limit", "conforming_loan_limit_desc", \n           "derived_loan_product_type", "derived_dwelling_category", "action_taken"]\none_hot += ["action_taken_desc", "purchaser_type", "purchaser_type_desc", "preapproval", \n            "preapproval_desc", "loan_type", "loan_type_desc"]                   \none_hot += ["loan_purpose", "loan_purpose_desc", "lien_status", "lien_status_desc", \n            "reverse_mortgage", "reverse_mortgage_desc"] \none_hot += ["open_end_line_of_credit", "open_end_line_of_credit_desc", "business_or_commercial_purpose"] \none_hot += ["negative_amortization", "negative_amortization_desc", "interest_only_payment"]\none_hot += [\'interest_only_payment_desc\', \'balloon_payment\',  \'balloon_payment_desc\']\none_hot += [\'other_nonamortizing_features\', \'other_nonamortizing_features_desc\']\none_hot += [\'construction_method\', \'construction_method_desc\', \'occupancy_type\']\none_hot += [\'multifamily_affordable_units\', \'debt_to_i

In [218]:
# Drop columns to be dropped
TRAIN=TRAIN.drop(drop_columns, axis=1)
TEST=TEST.drop(drop_columns, axis=1)

In [219]:
# Optionally, drop protected columns
# TRAIN=TRAIN.drop(protected_columns, axis=1)

In [220]:
# Apply one-hot encoding
for oh_column in one_hot:
    print(oh_column)
    oh_transformed_column = pd.get_dummies(TRAIN[oh_column])
    oh_transformed_column.columns = [oh_column + '_' + str(item) for item in list(oh_transformed_column.columns)]
    TRAIN = TRAIN.drop(oh_column, axis=1)
    TRAIN = pd.concat([TRAIN,oh_transformed_column], axis=1)
    
    oh_transformed_column = pd.get_dummies(TEST[oh_column])
    oh_transformed_column.columns = [oh_column + '_' + str(item) for item in list(oh_transformed_column.columns)]
    TEST = TEST.drop(oh_column, axis=1)
    TEST = pd.concat([TRAIN,oh_transformed_column], axis=1)
    

derived_loan_product_type
reverse_mortgage
negative_amortization
interest_only_payment
balloon_payment
applicant_credit_score_type
co_applicant_credit_score_type
applicant_age
co_applicant_age
applicant_age_above_62
co_applicant_age_above_62
derived_ethnicity
derived_race
derived_sex
debt_to_income_ratio


In [221]:
from sklearn.preprocessing import StandardScaler

features = ["loan_amount",  "loan_to_value_ratio", "discount_points",  "lender_credits"]
features += ["loan_term",  "prepayment_penalty_term",  "intro_rate_period"]
features += ["property_value", "income"]

scaler = StandardScaler()
scaler.fit(TRAIN[features])
std_features_TRAIN = pd.DataFrame(scaler.transform(TRAIN[features]), columns=[item + "_std" for item in features])
std_features_TEST = pd.DataFrame(scaler.transform(TEST[features]), columns=[item + "_std" for item in features])

TRAIN = pd.concat([TRAIN, std_features_TRAIN], axis=1)
TEST = pd.concat([TEST, std_features_TEST], axis=1)

#TRAIN = TRAIN.drop(features, axis=1)
#TEST = TEST.drop(features, axis=1)


/Users/kmontgomery/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/Users/kmontgomery/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  if __name__ == '__main__':
/Users/kmontgomery/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  # Remove the CWD from sys.path while we load stuff.


In [232]:
TRAIN.to_csv(output+'train_transformed.csv', index=False)
TEST.to_csv(output+'test_transformed.csv', index=False)

len_TRAIN = len(TRAIN)
len_TESTlen(TEST)

FULL = pd.concat([TRAIN, TEST], axis=0)
FULL.to_csv(output+'full_transformed.csv', index=False)

NameError: name 'len_TESTlen' is not defined

In [223]:
# Split into train and test file
# Add a cv column
import random

#TEST = TRAIN.iloc[int(len(TRAIN)*.7):len(TRAIN),:].copy()
#TRAIN = TRAIN.iloc[0:int(len(TRAIN)*.7),:].copy()

#TRAIN['fold'] = random.choices(list(range(0, 5)), k=len(TRAIN))

#TRAIN[target_encoded + [target_column, 'fold']].to_csv(output + "BLDS_one_hot_DATA.csv", index=False)
#TEST[target_encoded + [target_column]].to_csv(output + "BLDS_one_hot_TEST.csv", index=False)

In [224]:
#for enc_col in target_encoded:
#    TRAIN[enc_col] = TRAIN[enc_col].apply(str)
    
#TRAIN[target_column].unique()

In [225]:
"""import h2o
h2o.init()

blds_DATA = h2o.upload_file(main + "BLDS_one_hot_DATA.csv")

for enc_col in target_encoded:
    blds_DATA[enc_col] = blds_DATA[enc_col].asfactor()
    
blds_DATA[target_column] = blds_DATA[target_column].asfactor()
#blds_TEST = h2o.upload_file(main + "BLDS_one_hot_TEST.csv")"""

'import h2o\nh2o.init()\n\nblds_DATA = h2o.upload_file(main + "BLDS_one_hot_DATA.csv")\n\nfor enc_col in target_encoded:\n    blds_DATA[enc_col] = blds_DATA[enc_col].asfactor()\n    \nblds_DATA[target_column] = blds_DATA[target_column].asfactor()\n#blds_TEST = h2o.upload_file(main + "BLDS_one_hot_TEST.csv")'

In [226]:
"""
# Fit the target encoding map
from h2o.targetencoder import TargetEncoder
target_encoder = TargetEncoder(x=target_encoded, y=target_column,
                               fold_column="fold",
                               blended_avg= True,
                               inflection_point = 3,
                               smoothing = 1,
                               seed=1234)
target_encoder.fit(blds_DATA) """

'\n# Fit the target encoding map\nfrom h2o.targetencoder import TargetEncoder\ntarget_encoder = TargetEncoder(x=target_encoded, y=target_column,\n                               fold_column="fold",\n                               blended_avg= True,\n                               inflection_point = 3,\n                               smoothing = 1,\n                               seed=1234)\ntarget_encoder.fit(blds_DATA) '

In [227]:
TRAIN.iloc[0:3,:]


,loan_amount,loan_to_value_ratio,high_priced,discount_points,lender_credits,loan_term,prepayment_penalty_term,intro_rate_period,property_value,income,...,debt_to_income_ratio_Exempt,loan_amount_std,loan_to_value_ratio_std,discount_points_std,lender_credits_std,loan_term_std,prepayment_penalty_term_std,intro_rate_period_std,property_value_std,income_std
0,75000,95.000,0,1068.75,-1.0,360.0,-1.0,-1.0,75000.0,33.0,...,0,-0.860031,0.002212,0.359428,-0.306342,0.249762,-0.017041,-0.220705,-0.095474,-0.065176
1,545000,100.000,0,-1.00,-1.0,360.0,-1.0,-1.0,545000.0,298.0,...,0,1.153864,0.005063,-0.353662,-0.306342,0.249762,-0.017041,-0.220705,0.059397,0.117048
2,345000,68.082,0,-1.00,1735.0,360.0,-1.0,-1.0,515000.0,78.0,...,0,0.296887,-0.013136,-0.353662,1.159204,0.249762,-0.017041,-0.220705,0.049512,-0.034232


In [228]:
TRAIN.shape

(144000, 130)

In [229]:
list(TRAIN.columns)

['loan_amount',
 'loan_to_value_ratio',
 'high_priced',
 'discount_points',
 'lender_credits',
 'loan_term',
 'prepayment_penalty_term',
 'intro_rate_period',
 'property_value',
 'income',
 'cv_fold',
 'loan_to_value_ratio_is_missing',
 'discount_points_is_missing',
 'lender_credits_is_missing',
 'loan_term_is_missing',
 'prepayment_penalty_term_is_missing',
 'intro_rate_period_is_missing',
 'property_value_is_missing',
 'income_is_missing',
 'debt_to_income_ratio_is_missing',
 'derived_loan_product_type_Conventional:First Lien',
 'derived_loan_product_type_Conventional:Subordinate Lien',
 'derived_loan_product_type_FHA:First Lien',
 'derived_loan_product_type_FHA:Subordinate Lien',
 'derived_loan_product_type_FSA/RHS:First Lien',
 'derived_loan_product_type_FSA/RHS:Subordinate Lien',
 'derived_loan_product_type_VA:First Lien',
 'derived_loan_product_type_VA:Subordinate Lien',
 'reverse_mortgage_1',
 'reverse_mortgage_2',
 'reverse_mortgage_1111',
 'negative_amortization_1',
 'negative

In [230]:
sum(TRAIN['debt_to_income_ratio_is_missing'])

0

In [231]:
TRAIN['debt_to_income_ratio_is_missing'].iloc[0:20]

0     0
1     0
2     0
3     0
4     0
5     0
6     0
7     0
8     0
9     0
10    0
11    0
12    0
13    0
14    0
15    0
16    0
17    0
18    0
19    0
Name: debt_to_income_ratio_is_missing, dtype: int64